# Data

In [ ]:
from utils import *
config = load_config("config.yaml")

# read data source
df = pd.read_csv("data/batch_results.csv")
cols = ['mean_x', 'mean_y', 'sigma_x', 'sigma_y', 'emittance_x', 'emittance_y', 'transmission']
data = pick_from_list_str(df, cols, index=-1)
# convert all string to float
data = convert_strings_to_float(data)
# to log scale
data = log_scale_df(data, cols=['emittance_x', 'emittance_y', 'transmission'])
# remove outliers (thresholding percentile)
cols = ['mean_x', 'mean_y', 'sigma_x', 'emittance_y', 'transmission']
data = add_valid_flag(data, cols, 0.01)
data = data[data['valid'] == 1].drop(columns=['valid']).reset_index(drop=True)
# normalized range
data = normalize_min_max(data)
data

# Training

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf

# ——— Model ———

def build_model(config):
    inp = tf.keras.Input(shape=(len(config['input_cols']),))
    
    # 1st block
    x = tf.keras.layers.Dense(256, use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 2nd block
    x = tf.keras.layers.Dense(128, use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 3rd block
    x = tf.keras.layers.Dense(128, use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Dense(32, use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # linear output for full-range regression
    out = tf.keras.layers.Dense(len(config['target_cols']))(x)
    
    opt = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])
    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.Huber(),
        # loss='mse',
        metrics=['mae']
    )
    return model

model = build_model(config)
model.summary()

In [ ]:
# ——— Data prep ———
X = data[config['input_cols']].to_numpy(dtype='float32')
y = data[config['target_cols']].to_numpy(dtype='float32')

n = X.shape[0]
idx = tf.random.shuffle(tf.range(n), seed=config['seed'])
n_test = int(config['test_size'] * n)
n_val  = int(config['val_size']  * n)

test_idx  = idx[:n_test]
val_idx   = idx[n_test:n_test+n_val]
train_idx = idx[n_test+n_val:]

def make_ds(indices):
    return (tf.data.Dataset
              .from_tensor_slices((tf.gather(X, indices),
                                   tf.gather(y, indices)))
              .batch(config['batch_size'])
              .prefetch(tf.data.AUTOTUNE))

train_ds, val_ds, test_ds = map(make_ds, (train_idx, val_idx, test_idx))


# ——— Callbacks ———
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=config['patience'],
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=config['model_path'],
        monitor='val_loss',
        save_best_only=True
    )
]

# ——— Train ———
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config['epochs'],
    callbacks=callbacks,
    verbose=1
)
model.save(config['model_path'])
plot_history(history)

# ——— Eval ———
X_test = np.vstack([x for x, _ in test_ds])
y_test = np.vstack([y for _, y in test_ds])

# model predictions
y_pred = model.predict(test_ds)

# build and save dataframe
df_out = pd.DataFrame(
    np.hstack([X_test, y_test, y_pred]),
    columns=(
        config['input_cols']
      + [f"true_{c}" for c in config['target_cols']]
      + [f"pred_{c}" for c in config['target_cols']]
    )
)
df_out.to_csv(config['save_to'] + "test_results.csv", index=False)

# Evaluation

In [ ]:
from utils import *

result = pd.read_csv(config['save_to']+"test_results.csv")
result = clip_df(result)
process_df(result, inputs=config['input_cols'], outputs=config['target_cols'], save_to=config['save_to'])

# Exploratory data analysis

In [ ]:
from utils import *

config = load_config("config.yaml")
cols = ['mean_x', 'mean_y', 'sigma_x', 'sigma_y', 'emittance_x', 'emittance_y', 'transmission']


temp = []
for i in range(8):
    df = pd.read_csv("data/batch_results.csv")
    data = pick_from_list_str(df, cols, index=i)
    data = convert_strings_to_float(data)
    temp.append(data)
min_d, max_d = compute_bounds(temp)

temp = []
for i in range(8):
    df = pd.read_csv("data/batch_results.csv")
    data = pick_from_list_str(df, cols, index=i)
    data = convert_strings_to_float(data)
    temp.append(plot_distributions(data, bins=200, min_bounds=min_d, max_bounds=max_d, title=f'segment_{i+1}'))

figures_to_gif(temp, config['save_to'])